# Repertoire

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Mounted at /gdrive


In [ ]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np

# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

# Discogs data

## Load 

In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ "df_discogs_by_artist_tile.sav")[['master_id','artist',	'title']]
print('dimension  de la table: ' ,baseline.shape)
baseline['text'] =  baseline['artist'].astype(str) + ' ' +  baseline['title' ].astype(str)
baseline.drop(columns=['artist',	'title'], inplace=True)
baseline

## Normalize Baseline : Discogs data

In [ ]:
# from pandarallel import pandarallel

# Initialization
# pandarallel.initialize(progress_bar=True)

In [ ]:
start_time = datetime.now()

baseline["text_CLEAN"]= baseline.apply(lambda p : make_text_prep_func(p, name_column_blacklist, name_column_regex_replace,"text"),axis=1)

end_time = datetime.now()

baseline.to_pickle(REP_INTERMED+ 'baseline.sav')

print('Duration: {}'.format(end_time - start_time))

#Duration : 1h:53 min

Duration: 1:53:57.332362


In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ 'baseline.sav')
baseline.tail()

,master_id,text,text_CLEAN
8780041,0,･ﾟ✧(=✪ ᆺ ✪=)-･ﾟ✧ / Rainbow ★ Star ★ 36 Split,*; s *; rainbow star 36 split
8780042,0,￥Cuスタ平 サークル地主一派ゲーム曲ベスト,Y=cusutaPing sakuruDi Zhu Yi Pai gemuQu besuto
8780043,0,￥Cuスタ平 メイド・イン・ヤーパン,Y=cusutaPing meidoinyapan
8780044,0,￥Cuスタ平 / Nomico Runner Girl Bikininja,Y=cusutaPing nomico runn girl bikininj
8780045,0,��� ���,


## Vectorizer :  Keep TF-IDF strategy 


**Relativiser l’importance d’un terme dans un document
(TF) par son importance dans le corpus (IDF)**


+ Unlike TF-IDF, Word2Vec takes into account placement of words in a document(to some extent).Word2Vec is based on layyers and transformers

+ Bag of Words just creates a set of vectors containing the count of word occurrences in the document.

In [ ]:
start_time = datetime.now()

X_train, vectorizer = create_vectorizer_and_vectorized_db(baseline)

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

#Duration: 02 min:15s


**** creation du vectorizer


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


nb features: 1421716
**** sauvegarde des fichiers en pickle pour la prochaine fois
Duration: 0:02:07.446750


In [ ]:
X_train, vectorizer = reload_data_and_vectorizer()

# Query Data

In [ ]:
#import pandas as pd
#df = pd.read_excel(REP_INPUT +'2022-02-03_seller_reference_sample100K.xlsx')[['n_ref',	'artiste',	'titre']]
#print('dimension  de la table: ' ,df.shape)
#df['text'] =  df['artiste'].astype(str) +  ' ' + df['titre'].astype(str)
#df.drop(columns=['artiste',	'titre'], inplace=True)
#df.to_pickle(REP_INTERMED +'base_cdandlp.sav')
df= pd.read_pickle(REP_INTERMED +'base_cdandlp.sav')

In [ ]:
list_df = batch_taare(df, 1000)

# List Comprehension

## Execution (without multiprocessing)

In [ ]:
start_time = datetime.now()

temp = [ knn(df.iloc[i,1], vectorizer, X_train, baseline.copy(), 30)  for i in range(len(list_df[0])) ] 
      

end_time = datetime.now()


print("**** sauvegarde du  fichier en pickle pour la prochaine fois")
f = open(REP_INTERMED + 'results_sim2.pkl', 'wb')
pickle.dump(temp, f) 
f.close()


print('Duration: {}'.format(end_time - start_time))
# Duration: 1:20:28.829529


**** sauvegarde du  fichier en pickle pour la prochaine fois
Duration: 1:20:28.829529


## Parralize with joblib

In [ ]:
# number of cores
#import multiprocessing
#num_cores = multiprocessing.cpu_count()

#from joblib import Parallel, delayed
#results = Parallel(n_jobs=num_cores)(delayed(knn)(df.iloc[i,1], vectorizer, X_train, baseline.copy(), 30) for i in range(5))


# KNN unsupervised (sklearn api)

## Training with Manahattan

In [ ]:
nbrs = NearestNeighbors(n_neighbors=20, metric="manhattan",algorithm='brute').fit(X_train)

start_time = datetime.now()

result = [ nbrs.kneighbors(text_to_vect(df.iloc[i,1], vectorizer)) for i in range (len(list_df[0])) ]

f = open(REP_INTERMED + 'results_sim.pkl', 'wb')
pickle.dump(result, f) 
f.close()

end_time = datetime.now()


print('Duration: {}'.format(end_time - start_time))

#Duration: 0:46:45.063379


Duration: 0:13:05.634390


## Training with Euclidian

In [ ]:
nbrs = NearestNeighbors(n_neighbors=20,metric="euclidian",algorithm='brute').fit(X_train)

start_time = datetime.now()


result = [ nbrs.kneighbors(text_to_vect(df.iloc[i,1], vectorizer)) for i in range (len(list_df[0])) ]

end_time = datetime.now()


print('Duration: {}'.format(end_time - start_time))

#Duration: 0:46:45.063379

In [ ]:
## Load Output (Manathan)

In [ ]:
# All results
with open(REP_INTERMED + 'results_sim.pkl', 'rb') as file:
        results = pickle.load(file)

# extract one result :query  = 'bob marley uprising'

distances, indices = results[453]

temp = pd.concat(
          [ pd.DataFrame(distances).T, pd.DataFrame(indices).T ], 
          axis=1,
          )

temp.columns = ['distance','index']
temp

,distance,index
0,0.399467,814998
1,0.933777,815926
2,1.063252,814356
3,1.063252,814364
4,1.063252,814362
5,1.124852,814624
6,1.124852,814626
7,1.162382,814999
8,1.237242,814378
9,1.237242,814375


In [ ]:
distances, indices = results[453]
indexes = indices.flatten().tolist()
df_baseline = baseline.copy()
df_baseline['index'] = df_baseline.index

neighbors_sklearn_api= pd.merge(
                              df_baseline[df_baseline['index'].isin(indexes)], temp,
                              on = 'index',
                              how ='inner')

neighbors_sklearn_api.drop('index',axis=1,inplace=True)

neighbors_sklearn_api.to_excel(REP_INTERMED +'neighbors_sklearn_api.xlsx', index= False)
neighbors_sklearn_api

,master_id,text,text_CLEAN,distance
0,"2087887,1463321,335308,668989,0,1402697,221238...",Bob Marley Bob Marley,bob marley bob marley,1.063252
1,0,Bob Marley Bob Marley & Friends,bob marley bob marley friend,1.295980
2,1112183,Bob Marley Bob Marley 2,bob marley bob marley 2,1.063252
3,0,Bob Marley Bob Marley 3,bob marley bob marley 3,1.063252
4,0,Bob Marley Bob Marley And Friends,bob marley bob marley friend,1.295980
5,1064836,Bob Marley Bob Marley Collection,bob marley bob marley collect,1.276256
6,0,Bob Marley Bob Marley Live,bob marley bob marley liv,1.249344
7,0,Bob Marley Bob Marley Vol 2,bob marley bob marley vol 2,1.237242
8,0,Bob Marley Bob Marley Vol. 1,bob marley bob marley vol 1,1.237242
9,0,Bob Marley Bob Marley ‎– Vol. 1,bob marley bob marley - vol 1,1.237242


# Using map and partial

## Execution (without multiprocessing)

In [ ]:
from functools import partial


start_time = datetime.now()


func = partial(knn_scratch, vectorizer = vectorizer, X_train = X_train, df_items = baseline.copy(),nb_top=20)


answer = list(map(func,      [ list_df[0].iloc[i,1] for  i in range(len(list_df[0]))]))

f = open(REP_INTERMED + 'results_sim3.pkl', 'wb')
pickle.dump(answer, f) 
f.close()

end_time = datetime.now()


print('Duration: {}'.format(end_time - start_time))

NameError: ignored

## Execution (with multiprocessing)

Error 🇰
Failed To solve the issues: N

In [ ]:
#from functools import partial

# Execution with 8 cores  (all cores available in local)
#from multiprocessing import Pool
#pool = Pool(8)



#start_time = datetime.now()


#func = partial(knn_scratch, vectorizer = vectorizer, X_train = X_train, df_items = baseline.copy(),nb_top=20)


#answer = list(pool.map(func,      [ list_df[0].iloc[i,1] for  i in range(len(list_df[0]))]))

#f = open(REP_INTERMED + 'results_sim3.pkl', 'wb')
#pickle.dump(answer, f) 
#f.close()

#end_time = datetime.now()


#print('Duration: {}'.format(end_time - start_time))

## Load Output

In [ ]:
# All results
with open(REP_INTERMED + 'results_sim3.pkl', 'rb') as file:
        results = pickle.load(file)

# Export one example
results[458].to_excel(REP_INTERMED +'neighbors_knn_scratch.xlsx', index= False)
neighbors_sklearn_api

,master_id,text,text_CLEAN,distance
0,"2087887,1463321,335308,668989,0,1402697,221238...",Bob Marley Bob Marley,bob marley bob marley,1.063252
1,0,Bob Marley Bob Marley & Friends,bob marley bob marley friend,1.295980
2,1112183,Bob Marley Bob Marley 2,bob marley bob marley 2,1.063252
3,0,Bob Marley Bob Marley 3,bob marley bob marley 3,1.063252
4,0,Bob Marley Bob Marley And Friends,bob marley bob marley friend,1.295980
5,1064836,Bob Marley Bob Marley Collection,bob marley bob marley collect,1.276256
6,0,Bob Marley Bob Marley Live,bob marley bob marley liv,1.249344
7,0,Bob Marley Bob Marley Vol 2,bob marley bob marley vol 2,1.237242
8,0,Bob Marley Bob Marley Vol. 1,bob marley bob marley vol 1,1.237242
9,0,Bob Marley Bob Marley ‎– Vol. 1,bob marley bob marley - vol 1,1.237242
